In [1]:
#import neccessary libraries
import tensorflow as tf
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

2024-04-07 11:16:23.702364: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 11:16:23.702537: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 11:16:24.029817: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_df= pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
metadata_columns= ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
target_column= 'Pawpularity'
#Move last 500 images to test set
test_df= train_df.iloc[-500:]
#Delete last 500 images from train set
train_df= train_df.iloc[:-500]

#Now use last 500 images for validation
validation_df= train_df.iloc[-500:]
#Delete last 500 images from train set
train_df= train_df.iloc[:-500]

In [3]:
len(train_df), len(validation_df), len(test_df)

(8912, 500, 500)

In [4]:
#Get the images and metadata for the train set
def batch_generator(df, batch_size):
    while True:
        for i in range(0, len(df), batch_size):
            images= []
            metadata= []
            labels= []
            for j in range(i, min(i+batch_size, len(df))):
                image= tf.io.read_file('../input/petfinder-pawpularity-score/train/' + str(df['Id'].iloc[j]) + '.jpg')
                image= tf.image.decode_jpeg(image, channels=3)
                image= tf.image.resize(image, [256, 256])
                image= image/255.0
                images.append(image)
                metadata_image= df[metadata_columns].iloc[j].values
                metadata.append(metadata_image)
                label= df[target_column].iloc[j]/100.0
                labels.append(label)
            yield np.array(images), np.array(metadata), np.array(labels)

In [5]:
#Feature extraction using VGG16
base_model= tf.keras.applications.VGG16(include_top= False, weights= 'imagenet')
for layer in base_model.layers:
    layer.trainable= False
# train_image_features= base_model.predict(np.array(train_images))
# train_image_features= train_image_features.reshape(train_image_features.shape[0], -1)
# train_metadata= np.array(train_metadata)
# train_features_combined= np.concatenate((train_image_features, train_metadata), axis= 1)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# print(train_features_combined.shape)
# print(train_image_features.shape)

In [7]:
#Use XGBoost Regressor 
batch_size= 32
train_generator= batch_generator(train_df, batch_size)
validation_generator= batch_generator(validation_df, batch_size)
test_generator= batch_generator(test_df, batch_size)
xgb= XGBRegressor(learning_rate= 0.01, max_depth= 3, n_estimators= 200)
num_steps= len(train_df)//batch_size
for i in range(num_steps):
    images, metadata, labels= next(train_generator)
    print(images.shape, metadata.shape, labels.shape)
    train_image_features= base_model.predict(images)
    train_image_features= train_image_features.reshape(train_image_features.shape[0], -1)
    train_features_combined= np.concatenate((train_image_features, metadata), axis= 1)
    print(train_features_combined.shape)
    if(i==0):
        xgb.fit(train_features_combined, labels)
    else:
        xgb.fit(train_features_combined, labels, xgb_model= xgb)

(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 42s 42s/step


I0000 00:00:1712488648.737347      75 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


(32, 32780)
(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
(32, 32780)
(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
(32, 32780)
(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
(32, 32780)
(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
(32, 32780)
(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
(32, 32780)
(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
(32, 32780)
(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
(32, 32780)
(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
(32, 32780)
(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
(32, 32780)
(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
(32, 32780)
(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
(32, 32780)
(32, 256, 256, 3) (32, 12) (32,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
(32,

In [8]:
#Predict the pawpularity score for the test set
test_generator= batch_generator(test_df, batch_size)
test_predictions= []
test_labels= []
num_steps= len(test_df)//batch_size
file= open('predictions.txt', 'w')
for i in range(num_steps):
    images, metadata, labels= next(test_generator)
    test_image_features= base_model.predict(images)
    test_image_features= test_image_features.reshape(test_image_features.shape[0], -1)
    test_features_combined= np.concatenate((test_image_features, metadata), axis= 1)
    predictions= xgb.predict(test_features_combined)
    predictions = np.clip(predictions, 0, 1)
    test_predictions.append(predictions)
    test_labels.append(labels)
    file.write("Predictions\n")
    file.write(str(predictions))
    file.write("\n")
    file.write("True labels\n")
    file.write(str(labels))
    file.write("\n")

#Check the RMSE score
print("predictions", test_predictions)
print("True labels", test_labels)
rmse= np.sqrt(mean_squared_error(test_labels, test_predictions))
print('RMSE:', rmse)
file.close()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
predictions [array([1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1.,
       1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.],
      dtype=float32), array([0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1.,
       0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0.],
      dtype=float32), array([1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       0., 1

In [9]:
xgb.save_model('xgb_model.json')

In [10]:
# #Get the images and metadata for the testing set
# test_images= []
# test_metadata= []
# test_labels= []
# for i in range(5):
#     image= tf.io.read_file('petfinder-pawpularity-score/train/' + str(test_df['Id'].iloc[i]) + '.jpg')
#     image= tf.image.decode_jpeg(image, channels=3)
#     image= tf.image.resize(image, [256, 256])
#     image= image/255.0
#     test_images.append(image)
#     metadata= test_df[metadata_columns].iloc[i].values
#     test_metadata.append(metadata)
#     label= test_df[target_column].iloc[i]/100.0
#     test_labels.append(label)
  

In [11]:
# #Get testing image features
# test_image_features= base_model.predict(np.array(test_images))
# test_image_features= test_image_features.reshape(test_image_features.shape[0], -1)
# test_metadata= np.array(test_metadata)
# test_features_combined= np.concatenate((test_image_features, test_metadata), axis= 1)
# xgb.score(test_features_combined, test_labels)

# #Check the RMSE score
# predictions= xgb.predict(test_features_combined)
# print("predictions", predictions)
# print("True labels", test_labels)
# rmse= np.sqrt(mean_squared_error(test_labels, predictions))
# print('RMSE:', rmse)